In [1]:
! pip install pandas faiss-cpu sentence-transformers rank-bm25 scikit-learn transformers langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
! pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
from transformers import AutoTokenizer, BioGptForCausalLM
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
df = pd.read_csv("merged_medical_QA_dataset(2).csv").fillna("")
questions = df["question"].tolist()
answers = df["answer"].tolist()
sources = df["source"].tolist()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

chunked_answers = []
chunked_questions = []
chunked_sources = []

for i, text in enumerate(answers):
    chunks = text_splitter.split_text(text)
    chunked_answers.extend(chunks)
    chunked_questions.extend([questions[i]] * len(chunks))
    chunked_sources.extend([sources[i]] * len(chunks))

In [6]:
tokenized_corpus = [doc.split() for doc in chunked_answers]
bm25 = BM25Okapi(tokenized_corpus)

query = [
    "What are the symptoms of  breast cancer ?",
    "What should I do when I have a headache ?",
     "How does hypertension affect the heart?",
    "How can I see that I have skin cancer ?",
    "Is it safe to take paracetamol and ibuprofen together?",
    "What should I do if I experience chest pain ?",
    "What are the common symptoms of endometriosis?",
    "What are the signs of a seizure ?",
    "What are the best ways to manage chronic pain?",
    " I have a lump in my leg, What should I do?",
    "How to I know if I have food allergy?",
    "How long does it take to recover from a fractured tibia?",
    "What are the signs of a stroke?",
    "What are the early signs of pregnancy?",
    " Is an acute headache a symptom of brain cancer?",

    "Is ice effective for a sprained ankle ?",
    "How can I ease my period pain ?",
    "There is a black dot in my vision. Do I have eye cancer ?",
    " There is a yellow dot in my left eye. What it is ?",
    "I have a lump in my neck, should I be worried?",
    "At what age should a baby drink water?",
    "Why my heart and my left arm hurts ?",
    "Why does hurt when I pee?",
    "What causes intermittent headaches?",



    "What are the recommended guidelines for gestational diabetes screening and management?", #OK
    "How does hypertension contribute to the development of atrial fibrillation?",
    "How can we differentiate between essential tremor and Parkinson’s disease?",#OK
    "What are the indications for emergency C-section in labor?",#OK
    "How can we differentiate between bacterial and viral meningitis based on clinical and laboratory findings?",
    "How can we differentiate between an ischemic and a hemorrhagic stroke based on clinical presentation?",
    "What are the hemodynamic effects of propofol versus etomidate in induction of anesthesia?",

    "How often should I get a physical check-up?",
    "What vaccines do I need as an adult?",
    "Is it normal to feel tired all the time?",
    "What does it mean if my blood pressure is slightly high?",
    "How much water should I drink daily?",
    "What should I do if I have a persistent cough?",
    "When is a fever considered dangerous?",
    "What are the early signs of diabetes?",
    "How long should I take antibiotics for?",
    "Can I stop taking my medication if I feel better?",
    "Is it safe to take expired medication?",
    "When should I get a cholesterol test?",
    "Do I really need a flu shot every year?",
    "What can I do to strengthen my immune system?",
    "How can I lower my risk of heart disease?",
    "Is intermittent fasting safe ?",
    "How can I lose weight in a healthy way?",
    "What’s the best diet for managing high blood pressure?",
    "How much exercise should I get each week?",
    "Can too much screen time affect my sleep?",
    "How do I know if I have anxiety or just stress?",
    "What are the signs of depression?",
    "Can lack of sleep affect my mental health?",
    "Is it normal to feel unmotivated all the time?",
    "What birth control options are best for me?",
    "What are the signs of early menopause?",
    "Is it safe to take supplements during pregnancy?",
    "Should I be screened for prostate cancer?",
    "What can I do to improve testosterone levels naturally?",
    "Why do I feel tired even after 8 hours of sleep?",
    "Is hair loss preventable?",
    "How often should I get a colonoscopy?",
    "How can I manage my asthma better?",
    "What’s the best way to monitor my blood sugar?",
    "Can high cholesterol be managed without medication?",
    "How can I prevent my arthritis from getting worse?",
    "What is to long COVID?",
    "Should I get genetic testing for inherited diseases?",
    "How do I boost my energy levels naturally?",
    "Does stress affect physical health?",
    "Can gut health impact my mood?",
    "What are the warning signs of cancer I shouldn’t ignore?",
    "What should I do if I can't sleep at night?",
    "Is it normal to hear a ringing in my ears?",
    "How can I tell if a mole is dangerous?",
    "Are supplements like vitamin D necessary?",
    "Why do I feel dizzy when I stand up quickly?",
    "How long should a cold last?",
    "Is it okay to skip breakfast?",
    "Can allergies develop later in life?",
    "What is the difference between a cold and the flu?",
    "Should I be worried about numbness in my fingers?",
    "How do I know if I have high blood sugar?",
    "Can diet really affect my mental health?",
    "Why does my stomach hurt after eating?",
    "Can I exercise if I have joint pain?",
    "Is it safe to drink alcohol while taking antibiotics?",
    "What is the healthiest way to quit smoking?",
    "Why do I bruise easily?",
    "Can poor posture cause back pain?",
    "Is my shortness of breath normal during exercise?",
    "What are common side effects of birth control?",
    "Why do I feel anxious for no reason?",
    "Can I catch the same virus twice?",
    "How can I protect my vision as I age?",
    "Are probiotics actually beneficial?",
    "What is the difference between IBS and IBD?",
    "Can too much caffeine affect my heart?",
    "Should I be worried about my cholesterol if I’m slim?",
    "Why do I get leg cramps at night?",
    "What are the signs of an iron deficiency?",
    "Can I drink coffee if I have high blood pressure?",
    "Is it okay to take melatonin regularly?",
    "How can I reduce my risk of dementia?",
    "Why is my skin suddenly so dry?",
    "Can I still get pregnant after 40?",
    "What are normal blood sugar levels?",
    "Can dehydration cause headaches?",
    "How can I tell if I have a food intolerance?",
    "What vaccines do I need before traveling?",
    "How do I know if I need therapy?",
    "How do I manage seasonal allergies?",
    "Why do my hands tingle sometimes?",
    "Can a lack of exercise cause health problems?",
    "Should I be using sunscreen every day?",
    "Is it normal to forget things more often with age?",
    "Can long-term screen time affect my eyesight?"

]

k = 10  # top-k documents

# === Step 3: Dense Retrieval with BioBERT + FAISS ===
embedding_model = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
corpus_embeddings = embedding_model.encode(chunked_answers, show_progress_bar=True)
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2331 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [7]:
# === Step 4: Hybrid Retrieval via RRF ===
def reciprocal_rank_fusion(rank_lists, k=60):
    scores = {}
    for ranking in rank_lists:
        for rank, idx in enumerate(ranking):
            scores[idx] = scores.get(idx, 0) + 1 / (k + rank + 1)
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

In [8]:
retrieved_docs = []
retrieved_sources = []
retrieved_scores = []

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
top_n=3
for queries in query:
    # Sparse retrieval
    bm25_scores = bm25.get_scores(queries.split())
    bm25_topk_idx = np.argsort(bm25_scores)[::-1][:k]

    # Dense retrieval
    query_embedding = embedding_model.encode([queries])
    dense_scores, dense_topk_idx = index.search(query_embedding, k)
    dense_topk_idx = dense_topk_idx[0]

    # RRF Fusion
    fused = reciprocal_rank_fusion([bm25_topk_idx, dense_topk_idx])[:k]
    fused_indices = [idx for idx, _ in fused]

    # Rerank
    rerank_inputs = [(queries, chunked_answers[i]) for i in fused_indices]
    rerank_scores = reranker.predict(rerank_inputs)
    reranked = sorted(zip(rerank_scores, fused_indices), reverse=True)[:3]

    context = "\n".join([f"Q: {chunked_questions[j]}\nA: {chunked_answers[j]}" for _, j in reranked])
    srcs = list(set([chunked_sources[j] for _, j in reranked]))
    scores_only = [s for s, _ in reranked]

    norm_scores = MinMaxScaler().fit_transform(np.array(scores_only).reshape(-1, 1)).flatten()

    retrieved_docs.append(context)
    retrieved_sources.append(srcs)
    retrieved_scores.append(norm_scores)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packa

In [9]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt-large")
biogpt = BioGptForCausalLM.from_pretrained("microsoft/biogpt-large").to("cuda" if torch.cuda.is_available() else "cpu")

prompts = [
    f"Context: {retrieved_docs[i]}\n\n"
    f"According to the information available in the context and the knowledge you already have, "
    f"answer the following question:\n{query[i]}\n\nAnswer:"
    for i in range(len(query))
]

answers = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(biogpt.device)
    output = biogpt.generate(**inputs, max_new_tokens=128)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    answers.append(decoded)

# === Output Final Answers ===
for i, (q, a) in enumerate(zip(query, answers)):
    print(f"\nQuestion: {q}\nAnswer: {a}\nSources: {', '.join(retrieved_sources[i])}")
    print("Relevance Scores:")
    for score in retrieved_scores[i]:
        print(f"- {score:.4f}")
    print("=" * 50)


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.29G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]


Question: What are the symptoms of  breast cancer ?
Answer: Context: Q: What is (are) Breast Cancer? A:. Symptoms of breast cancer may include a lump in the breast, a change in size or shape of the breast or discharge from a nipple. Breast self-exam and mammography can help find breast cancer early when it is most treatable. Treatment may consist of radiation, lumpectomy, mastectomy, chemotherapy and hormone therapy. Men can have breast cancer, too, but the number of cases is small. NIH: National Cancer Institute Q: What are the symptoms of BRCA1 hereditary breast and ovarian cancer syndrome? A: What are the signs and symptoms of BRCA1 hereditary breast and ovarian cancer syndrome? The Human Phenotype Ontology provides the following list of signs and symptoms for BRCA1 hereditary breast and ovarian cancer syndrome. If the information is available, the table below includes how often the symptom is seen in people with this condition. You can use the MedlinePlus Medical Dictionary to loo

In [10]:
import json
tout2=[]
for q,r in zip(query,answers):
    data_json_biogpt_RAG = {"question": q, "answer": r}
    tout2.append(data_json_biogpt_RAG)
with open("biogpt_RAG_chunking.json", "w", encoding="utf-8") as f:
           json.dump(tout2, f, ensure_ascii=False, indent=4)